In [ ]:
from dask.distributed import Client, LocalCluster
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from tqdm import tqdm
from IPython import display
import importlib
import utils
import velocity_helpers

In [ ]:
import importlib
importlib.reload(velocity_helpers)

In [ ]:
cluster = LocalCluster()
client = Client(cluster)
print(client.dashboard_link)

In [ ]:
## read in lines; lazily get velocity cubes, centrelines, and robust trends
lines = gpd.read_file('../data/streams_v2.geojson')
V = {}
failed = []
for row in tqdm(lines.sample(1).itertuples()):
    print(f'working on #{row.Index}')
    # try:
    V[row.Index] = velocity_helpers.CentreLiner(
        geo=row.geometry,
        buff_dist=3_000,
        index=row.Index,
        filter_cube=False,
        get_robust_trend=True,
        get_annual_median=False,
        sample_centreline=False,
        get_rgb=False,
        **{'ddt_range': ('335d', '395d'),
            'robust_trend_export': True})
        # display.clear_output(wait=True)
    # except Exception as e:
    #     failed.append((row.Index, e))
    #     print(f'#{row.Index} did not work because\n{e}')
    #     continue


In [ ]:
old = xr.open_dataset(
    '../results/intermediate/velocity/robust_annual_trends/id10.zarr/',
    engine='zarr'
)
new = xr.open_dataset(
    '../results/intermediate/velocity/robust_annual_trends/240524_2312_id10.zarr/',
    engine='zarr')



In [ ]:
fig, axs = plt.subplots(ncols=3, sharex=True, sharey=True, subplot_kw={'aspect':'equal'})

old['v_trend'][:,:,0].plot(ax=axs[0], vmin=-20, vmax=20, cmap='RdBu_r')
new['v_trend'][:,:,0].plot(ax=axs[1], vmin=-20, vmax=20, cmap='RdBu_r')
(new['v_trend'][:,:,0] - old['v_trend'][:,:,0]).plot(ax=axs[2], vmin=-2, vmax=2, cmap='PuOr')

In [ ]:

# compute robust trends and export at the same time
for k, v in tqdm(V.items()):
    v.robust_trend['v_trend'].attrs = {'crs':3413,
                                       'buffer':3000,
                                       'ddt_range': v.ddt_range,
                                       'mad_n': v.n,
                                       'date':str(pd.Timestamp.now()),
                                       'centreline':v.tidy_stream.wkt,
                                       'centreline_id': k}
    
    (v.robust_trend['v_trend']
     .chunk(dict(zip(v.robust_trend['v_trend'].dims,
                    v.robust_trend['v_trend'].shape)))
     .to_zarr(f'../results/intermediate/velocity/robust_annual_trends/id{k}.zarr')
     )


In [ ]:
import pandas as pd

In [ ]:
pd.Timestamp.now().strftime('%Y%m%d_%H%M')

In [ ]:
pd.Timestamp.now()now = pd.Timestamp.now().strftime('%y%m%d_%H%M')+ '_'
directory = ''
filename = '.png'
fig.savefig(directory + now + filename, bbox_inches = 'tight')

# plotting

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob

In [ ]:
lines = gpd.read_file('../data/streams_v2.geojson')

In [ ]:
files = glob('../results/intermediate/velocity/robust_annual_trends/*.zarr')
dss = [xr.open_dataset(f, engine='zarr')['v_trend'] for f in files]

In [ ]:
ids = [ds.attrs['centreline_id'] for ds in dss]

In [ ]:
results= dict(zip(ids, dss))

In [ ]:
from shapely import wkt

In [ ]:
results.keys()

In [ ]:
fig, axs = plt.subplots(ncols=3,
                        nrows=5,
                        subplot_kw={'aspect':'equal'},
                        figsize=[15,15])

# for k, v in results.items():
i = 0
for row in lines.itertuples(): 
    try:
        v = results[row.Index]
    except KeyError:
        continue
    
    v[:,:,0].plot(ax=axs.flat[i], robust=True, cmap='RdBu_r', cbar_kwargs={'label':'m yr^-2'})    
    _p = utils.shapely_reprojector(row.geometry.boundary.geoms[0])
    axs.flat[i].set_title(f'{_p.y:.2f} N, {-_p.x:.2f} W', y=0.97)
    axs.flat[i].plot(*row.geometry.coords.xy, c='k', ls=':')
    axs.flat[i].set_axis_off()
    i += 1

[ax.remove() for ax in axs.flat[-2:]]
# [axs.flat[q].remove() for q in [i for i in range(len(axs.flat)) if i not in results.keys()]]
fig.savefig('../results/intermediate/velocity/robust_annual_trends/spatial_plot.png')

now = pd.Timestamp.now().strftime('%y%m%d_%H%M')+ '_'
directory = '../../GitHub/diary/journal_figures/'
filename = 'robust_annual_trends_spatial_plot.png'
fig.savefig(directory + now + filename, bbox_inches = 'tight')

In [ ]:
fig, axs = plt.subplots(ncols=3, nrows=5)

ls = wkt.loads(results[1].attrs['centreline'])
ax.plot(*ls.coords.xy, c='k')